In [ ]:
# requirements
!pip install -q -U \
    transformers==4.56.2 \
    datasets==4.1.1 \
    accelerate==1.10.1 \
    peft==0.17.1 \
    trl==0.23.0 \
    bitsandbytes==0.48.0 \
    triton==3.4.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.6/503.6 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 17.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.


In [ ]:
import os
os.environ["WANDB_MODE"] = "disabled"

In [ ]:
from huggingface_hub import login
login()

## loading data


In [ ]:
import json
from datasets import load_dataset


dataset_id = "ronit-26/project-docs"

dataset = load_dataset(dataset_id, split="train")

# formating the datapoint in the input and output with input doc containing a prompt for instruction
def format_prompt(example):
    prompt = f"""<|start_header_id|>user<|end_header_id|>

**Analyze the following project document to identify the key tasks, dependencies, and requirements. Your goal is to interpret the author's intent, even if the document is informal or contains minor errors.

Based on your analysis, generate a structured list of actionable GitHub issues in JSON format. Follow these principles:**
**- Decompose large goals into smaller, atomic tasks.**
**- Infer logical labels (e.g., 'frontend', 'backend', 'database') based on the task description.**
**- Infer dependencies from the logical workflow if they are not explicitly stated.**

**Project Document:**
{example['doc']}

<|eot_id|><|start_header_id|>assistant<|end_header_id|>
{json.dumps(example['issues'])}<|eot_id|>"""
    example['text'] = prompt
    return example

# Apply the formatting function
dataset = dataset.map(format_prompt)

# Split the dataset for training and evaluation
dataset_split = dataset.train_test_split(test_size=0.1, seed=42)

train_dataset = dataset_split["train"]
eval_dataset = dataset_split["test"]

# Check the size of your datasets
print(f"Dataset loaded successfully from '{dataset_id}'")
print(f"Training dataset size: {len(train_dataset)}")
print(f"Evaluation dataset size: {len(eval_dataset)}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Resolving data files:   0%|          | 0/457 [00:00<?, ?it/s]

Dataset loaded successfully from 'ronit-26/project-docs'
Training dataset size: 411
Evaluation dataset size: 46


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Base model that we want to fine tune on from hugging face.
# model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
model_id="ronit-26/finetuned-issues-generator"

In [ ]:
# A configuration object for 4-bit quantization `QLoRA`....(for fitting the model into the GPU's memory)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, # to load the model's weights in 4-bit precision
    bnb_4bit_quant_type="nf4",# Normal Float 4
    bnb_4bit_compute_dtype=torch.bfloat16 # While the model is stored in 4-bit, the actual math during training is performed in a higher 16-bit precision
)

In [ ]:
# Load the pre-trained model from the Hub with our quantization settings.
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto" # Automatically places the model on the GPU
)

# Loads the tokenizer associated with the Llama 3 model.
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [ ]:
from peft import LoraConfig
from trl import SFTConfig
from transformers import TrainingArguments

# configuration object for the LoRA adapters
lora_config = LoraConfig(
    r=16, # rank of matrices for training
    lora_alpha=32, # scaling factor
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# configuration object for all the training hyperparameters
training_args = SFTConfig(
    output_dir="./results",
    # --- Training Hyperparameters ---
    num_train_epochs=1,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    logging_steps=10,
    # --- Other Trainer Settings ---
    save_strategy="epoch",
    eval_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
    hub_model_id="ronit-26/finetuned-issues-generator",
    hub_strategy="every_save",
    report_to=None,
    # --- SFT Specific Settings ---
    max_length=2048,
    dataset_text_field="text",
)

In [ ]:
from trl import SFTTrainer

# SFTTrainer object, which orchestrates the entire fine-tuning process
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=lora_config,
    processing_class=tokenizer,
    args=training_args,
)
trainer.train()

/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:


Abort: 